In [1]:
import tensorflow
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K


In [2]:
from keras.datasets import mnist
from matplotlib import pyplot
# load dataset
(trainX, trainy), (testX, testy) = mnist.load_data()
# summarize loaded dataset

In [3]:
# LOAD DATA
from tensorflow.keras.utils import to_categorical 
from sklearn.model_selection import train_test_split


# PREPROCESSING
def preprocess_mnist(x_train, y_train, x_test, y_test):
    # Normalizing all images of 28x28 pixels
    x_train = x_train.reshape(-1, 28, 28, 1)
    x_test = x_test.reshape(-1, 28, 28, 1)
    input_shape = (28, 28, 1)
    
    # Float values for division
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    
    # Normalizing the RGB codes by dividing it to the max RGB value
    x_train /= 255
    x_test /= 255
    
    # Categorical y values
    y_train = to_categorical(y_train)
    y_test= to_categorical(y_test)
    
    return x_train, y_train, x_test, y_test, input_shape
    
X_train, y_train, X_test, y_test, input_shape = preprocess_mnist(trainX, trainy, testX, testy)

In [4]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import  Conv2D, MaxPooling2D, Dropout, Flatten, Dense,GlobalAveragePooling2D
def build_cnn(activation = 'relu',
              dropout_rate = 0.2,
              optimizer = 'Adam'):
    model = Sequential()
    
    model.add(Conv2D(32, kernel_size=(3, 3),
              activation=activation,
              input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation=activation))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_rate))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(128, activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(10, activation='softmax'))
    
    model.compile(
        loss='categorical_crossentropy', 
        optimizer=optimizer, 
        metrics=['accuracy']
    )
    
    return model

In [5]:
from keras.backend import set_session
from keras.backend import clear_session
from keras.backend import get_session
import gc

# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del classifier # this is from global space - change this as you need
    except:
        pass

    print(gc.collect()) # if it does something you should see a number as output

    # use the same config as you used to create the session
    config = tensorflow.compat.v1.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    set_session(tensorflow.compat.v1.Session(config=config))

In [6]:
def algorithm_pipeline(X_train_data, X_test_data, y_train_data, y_test_data, 
                       model, param_grid, cv=5,
                       do_probabilities = False):
    
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid, 
        cv=cv, 
        n_jobs=1, 
        
        verbose=2
    )
    gs.fit(X_train_data, y_train_data)
    
    if do_probabilities:
        pred = gs.predict_proba(X_test_data)
    else:
        pred = gs.predict(X_test_data)
    
    reset_keras()
    return gs

In [22]:
from sklearn.model_selection import train_test_split, GridSearchCV
param_grid = {
              'epochs':[20,30],
              'batch_size':[200]
              #'epochs' :              [100,150,200],
              #'batch_size' :          [32, 128],
              #'optimizer' :           ['Adam', 'Nadam'],
              #'dropout_rate' :        [0.2, 0.3],
              #'activation' :          ['relu', 'elu']
             }

model = KerasClassifier(build_fn = build_cnn, verbose=0)
reset_keras()
gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid, 
        cv=2, 
        n_jobs=1, 
        verbose=2
    )
reset_keras()




10452
101


In [23]:
gs.fit(X_train, y_train)

Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV] END ..........................batch_size=200, epochs=20; total time=  58.7s
[CV] END ..........................batch_size=200, epochs=20; total time= 1.0min
[CV] END ..........................batch_size=200, epochs=30; total time= 1.6min
[CV] END ..........................batch_size=200, epochs=30; total time= 1.6min


GridSearchCV(cv=2,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x00000212B850A5B0>,
             n_jobs=1, param_grid={'batch_size': [200], 'epochs': [20, 30]},
             verbose=2)

In [24]:
print(gs.best_score_)
print(gs.best_params_)

0.9075166583061218
{'batch_size': 200, 'epochs': 30}
